In [1]:
from nltk.tokenize import word_tokenize
import pandas as pd
import spacy 
import matplotlib.pyplot as plt
import gzip,os,glob
import numpy as np
import pickle
import nltk
import codecs
from langdetect import detect

In [2]:
# load data from pkl file 
df = pd.read_pickle('../data/data_filtered contents.pkl')
print(df.shape)

(20247, 5)


In [ ]:
# Adds new collumn, detects language of column
df['Languagereview'] = df['content'].apply(detect)

In [ ]:
# delets all rows with the value "en"
df = df[df.Languagereview == "de"]

In [ ]:
# resets index from 0, 4, 11, ... (random) to 0,1,2, ...
df.reset_index(inplace=True,drop=True)
print(df.shape)

In [ ]:
df_movies = pd.read_pickle('../data/df_movies_cleaning_2.pkl')
df_movies

In [ ]:
# Removing non-alphabetical symbols in dataframe, keeping umlauts
df_movies['content'] = df_movies['content'].replace('[^a-zA-Z \äöüß]', '', regex=True).str.replace(' +',' ').str.strip()
# Transform umlauts
df_movies['content'] = df_movies['content'].str.replace(chr(252), 'ue')
df_movies['content'] = df_movies['content'].str.replace(chr(228), 'ae')
df_movies['content'] = df_movies['content'].str.replace(chr(246), 'oe')
df_movies['content'] = df_movies['content'].str.replace(chr(196), 'Ae')
df_movies['content'] = df_movies['content'].str.replace(chr(214), 'Oe')
df_movies['content'] = df_movies['content'].str.replace(chr(220), 'Ue')
df_movies['content'] = df_movies['content'].str.replace(chr(223), 'ss')
# Lowercase
df_movies['content'] = df_movies['content'].str.lower()
# rename collumn
df_movies.rename(columns={'title1': 'title'}, inplace=True)
df_movies.rename(columns={'title3': 'author'}, inplace=True)
# drop collumn
df_movies.drop(['word_count'],1,inplace=True)
df_movies.drop(['title2'],1,inplace=True)
# add label collumn
df_movies['medium'] ='movie'
# add nan
df_movies.fillna(value=pd.np.nan, inplace=True)
df_movies.head()

In [ ]:
# Removing non-alphabetical symbols in dataframe, keeping umlauts
df['content'] = df['content'].replace('[^a-zA-Z \äöüß]', '', regex=True).str.replace(' +',' ').str.strip()
# Transform umlauts
df['content'] = df['content'].str.replace(chr(252), 'ue')
df['content'] = df['content'].str.replace(chr(228), 'ae')
df['content'] = df['content'].str.replace(chr(246), 'oe')
df['content'] = df['content'].str.replace(chr(196), 'Ae')
df['content'] = df['content'].str.replace(chr(214), 'Oe')
df['content'] = df['content'].str.replace(chr(220), 'Ue')
df['content'] = df['content'].str.replace(chr(223), 'ss')
# Lowercase
df['content'] = df['content'].str.lower()
# add label collumn
df['medium'] ='print'
df.head()

<h1>Combining content</h1>

In [ ]:
df['title'] = df['title'].str.split('  / von').str[0]
df['title'] = df['title'].str.replace('[^\w\s]','')
df['title'] = df['title'].replace('\s+', ' ', regex=True)
df['title'] = df['title'].str.lower()

In [ ]:
df['author'] = df['author'].str.split('  / von').str[0]
df['author'] = df['author'].str.replace('[^\w\s]','')
df['author'] = df['author'].replace('\s+', ' ', regex=True)
df['author'] = df['author'].str.lower()

In [ ]:
df_combined = df.copy()
df_combined.drop(['class','word_count', 'Languagereview'],axis=1, inplace=True)
df_combined.replace(np.nan, '', regex=True, inplace=True)
df_combined.replace('\s+', ' ', regex=True, inplace=True)

df_combined['content_nltk'] = df_combined['title'] +" "+ df_combined['content']
df_combined['content_nltk'] = df_combined['content_nltk'].str.strip()

df_movies['title'] = df_movies['title'].str.lower()

df_movies['content_nltk'] = df_movies['title'] +" "+ df_movies['content']
df_movies['content_nltk'] = df_movies['content_nltk'].str.strip()

# df['content_nltk'] = df_combined
# df_movies['content_nltk'] = df_movies_combined

# whitespaces to nan
df_combined = df_combined.replace(r'^\s*$', np.nan, regex=True)

In [ ]:
df_combined.replace('\s+', ' ', regex=True, inplace=True)
df_combined = pd.concat([df_combined, df_movies])
df_combined.reset_index(inplace=True,drop=True)
df_combined = df_combined[['title','author','content_nltk','medium']]
df_combined

<h1>Remove Stop Words</h1>

In [49]:
from nltk.corpus import stopwords
stop = stopwords.words('german')

df_combined['content_nltk'] = df_combined['content_nltk'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_combined['content_nltk'].iloc[3]

'grüne philosophie konservativer denkansatz umweltschutz liegt herkoemmlich hoheitsgebiet politischen linken bedrohung natur zufolge internationalen kapitalismus konsumverhalten unbegrenzten ausbeutung natuerlicher ressourcen zuzuschreiben roger scruton entbloesst grundverstaendnis gefaehrlichen trugschluss zerstoererisch kosysteme wirkt denen zukunft abhaengt konservative denker wuerdigt traditionelle prinzipien offensichtlichste wirksamste mittel erhalt planeten sichern lokalismus buergerverantwortung schlagen dabei bemuehungen schwerfaelliger nichtregierungsorganisationen internationaler komitees zukunft mitnichten gesichert roger scruton beweist gedankenreichen schrift fortbestand kosystems erde moeglich'

<h1>Tokenizing</h1>

In [50]:
df_tok = df_combined['content_nltk']
df_tok

0        aernout mik communitas aernout mik zaehlt wich...
1        anton henning dt engl vorliegende publikation ...
2        feuchtwangers familie tradition jüdisches selb...
3        grüne philosophie konservativer denkansatz umw...
4        technische dokumentation maschinen anlagenbau ...
                               ...                        
25579    jochen roller let's dance! lecture performance...
25580    ai weiwei - stimmen verstummen chinesische kue...
25581    new york tomorrow webdokumentation new york gi...
25582    hostess hostess seit zwei jahren automechanike...
25583    bruno peinado bruno peinado plastische kunst r...
Name: content_nltk, Length: 25584, dtype: object

In [81]:
tokenized = [ nltk.word_tokenize( str(sentence), language='german' ) for sentence in df_tok ]
tokenized_test = tokenized[0:5]
print(tokenized_test)

[['aernout', 'mik', 'communitas', 'aernout', 'mik', 'zaehlt', 'wichtigsten', 'kuenstlern', 'niederlande', 'installationen', 'film', 'video', 'performance', 'skulptur', 'architektur', 'oszillieren', 'kunst', 'bewegten', 'bild', 'grundlegend', 'weiterentwickelt', 'werke', 'spiegeln', 'gesellschaften', 'kriege', 'wirtschaftliche', 'depression', 'wider', 'katalog', 'praesentiert', 'wichtige', 'arbeiten', 'kuenstlers', 'nie', 'zuvor', 'deutschland', 'sehen', 'ausstellung', 'oktober', 'januar', 'museum', 'folkwang', 'essen'], ['anton', 'henning', 'dt', 'engl', 'vorliegende', 'publikation', 'widmet', 'drei', 'ortsbezogenen', 'installationen', 'anton', 'henning', 'allesamt', 'drei', 'exponierten', 'orten', 'kunst', 'ausgefuehrt', 'frankfurter', 'salon', 'hans', 'hollein', 'erbauten', 'museum', 'fuer', 'moderne', 'kunst', 'mmk', 'frankfurt', 'main', 'oktogon', 'fuer', 'herford', 'frank', 'gehry', 'erbauten', 'erst', 'kuerzlich', 'eroeffneten', 'marta', 'herford', 'apotheotischen', 'antiphrasen'

In [53]:
# df_tok = df.apply(lambda row: nltk.word_tokenize(row['content_nltk'], language='german'), axis=1)

In [176]:
df_tok

In [177]:
# df_tok_test = df_tok[:3]
# df_tok_test

<h1>Lemmetization</h1>

In [77]:
import nltk
import codecs

for i in tokenized:
    for word in i:
        word = nltk.sent_tokenize(w,language='german')

# tokenized_sent = nltk.tokenize.word_tokenize(text,language='german')
# print(tokenized_sent)

AttributeError: 'Series' object has no attribute 'iterrows'

In [89]:
from HanTa import HanoverTagger as ht

tagger = ht.HanoverTagger('morphmodel_ger.pgz')

In [101]:
new = []
for items in tokenized_test:
  new.append([])
  for item in items:
     item = tagger.tag_sent(tokenized_test)
     new[-1].append(item)


# tags = tagger.tag_sent(tokenized_test)
# print(tags)

TypeError: expected string or bytes-like object

In [46]:
words = [] 
sentences_tok = [nltk.tokenize.word_tokenize(sent) for sent in sentences]
for sent in sentences_tok:
    tags = tagger.tag_sent(sent) 
    nouns_from_sent = [lemma for (word,lemma,pos) in tags if pos != "ZY"]
    nouns.extend(nouns_from_sent)

In [47]:
print(words)

['unser', 'Text', 'sein', 'nicht', 'sehr', 'lang', '--', 'aber', 'zum', 'Schluß', 'schreiben', 'wir', 'doch', 'noch', 'mal', 'etwas', 'mehr', 'Code', 'und', 'machen', 'noch', 'mal', 'eine', 'nett', 'Statistik']
['sein', 'schreiben', 'machen']


In [178]:
import spacy
nlp = spacy.load("de_core_news_sm")

df_combined["content_nltk"] = df_combined["content_nltk"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [179]:
df_combined["content_nltk"]

0        aernout mik communitas aernout mik zaehlt wich...
1        anton henning dt engl vorliegend publikation w...
2        feuchtwangers familie tradition jüdisch selbst...
3        grüne philosophie konservativ denkansatz umwel...
4        technische dokumentation maschinen anlagenbau ...
                               ...                        
25584    jochen roller let 's dance ! lecture performan...
25585    ai weiwei - stimmen verstummen chinesische kue...
25586    new york tomorrow webdokumentation new york ge...
25587    hostess hostess seit zwei jahren automechanike...
25588    bruno peinado bruno peinado plastische kunst r...
Name: content_nltk, Length: 25589, dtype: object

<h1>Export</h1>

In [ ]:
df_import = pd.read_pickle('../data/data_filtered contents.pkl')

#df_combined['content'] = 

In [33]:
df.to_pickle('../data/after_nltk/data_movies_after_nltk.pkl')
# df_content.to_csv('../data/after_nltk/df_content.csv', index=True)
#df["content_nltk"].to_csv('../data/after_nltk/df_content_after_nltk+titles.csv', index=True)
df["content_nltk"].to_csv('../data/after_nltk/df_movies_content_after_nltk.csv', index=True)
# df.to_excel('../data/after_nltk/data_after_nltk.xlsx', index = False)

In [34]:
# #counts number of lines
# fname = "df_content.txt"
# count = 0
# with open(fname, 'r') as f:
#     for line in f:
#         count += 1
# print("Total number of lines is:", count)